In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features
from src.visualization import visualize

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# Lectura de bases de datos

En este análisis se utilizarán 2 bases de datos:

- Base de datos de consultas ambulatorias INT
- Base de datos de procedimientos ambulatorios INT

Ambas bases de datos tienen fechas distintas. 

In [ ]:
cie = pd.read_excel("../data/external/CIE-10 - sin_puntos_y_X.xlsx")

In [ ]:
df_diagnosticos = pd.read_csv(
    "../data/processed/datos_limpios_diagnosticos.csv", encoding="latin-1", sep=";"
)
df_diagnosticos = build_features.formatear_fechas_ambulatorio(df_diagnosticos)
df_diagnosticos = build_features.agregar_rango_etario(df_diagnosticos)
df_diagnosticos = build_features.agregar_anio_mes_dia(df_diagnosticos, "fecha_atencion")
df_diagnosticos = df_diagnosticos.sort_values("fecha_atencion")

In [ ]:
df_procedimientos = pd.read_csv(
    "../data/processed/datos_limpios_procedimientos.csv", encoding="latin-1", sep=";"
)
df_procedimientos = build_features.formatear_fechas_procedimientos(df_procedimientos)
df_procedimientos = build_features.agregar_anio_mes_dia(df_procedimientos, "fecha_realizacion")
df_procedimientos = df_procedimientos.sort_values("fecha_realizacion")

cols_texto = df_procedimientos.select_dtypes(include="object").apply(lambda x: x.str.strip())
df_procedimientos.loc[:, cols_texto.columns] = cols_texto

## 1. Analisis de cantidad de consultas y procedimientos

En este apartado se quieren responder las siguientes preguntas:

- ¿Cuál es el rango temporal en la que están ambas bases de datos analizadas?
- ¿Cuántas consultas hay?
- ¿Cuántos procedimientos hay?

In [ ]:
fecha_min_diags = min(df_diagnosticos.fecha_atencion)
fecha_max_diags = max(df_diagnosticos.fecha_atencion)

fecha_min_proced = min(df_procedimientos.fecha_realizacion)
fecha_max_proced = max(df_procedimientos.fecha_realizacion)

print(f"El rango de fechas de diagnosticos es entre {fecha_min_diags} y " f" {fecha_max_diags}")
print(f"El rango de fechas de procedimientos es entre {fecha_min_proced} y " f" {fecha_max_proced}")

print(f"\nLa base de datos de diagnosticos tiene {df_diagnosticos.shape[0]} consultas")
print(f"La base de datos de procedimientos tiene {df_procedimientos.shape[0]} procedimientos")

sns.histplot(df_diagnosticos["fecha_atencion"])
sns.histplot(df_procedimientos["fecha_realizacion"])
plt.show()

Los resultados anteriores indican que ambas bases están en rangos temporales distintos. Sin embargo,
la base de procedimientos está contenida temporalmente en la base de consultas ambulatorias.

Para efectos comparativos, se filtrará la base de consultas ambulatorias para que esté en el mismo
rango temporal que la base de procedimientos.

In [ ]:
mask_fechas = (df_diagnosticos["fecha_atencion"] >= fecha_min_proced) & (
    df_diagnosticos["fecha_atencion"] <= fecha_max_proced
)

df_diagnosticos_acotado = df_diagnosticos[mask_fechas]

In [ ]:
fecha_min_diags_acotada = min(df_diagnosticos_acotado.fecha_atencion)
fecha_max_diags_acotada = max(df_diagnosticos_acotado.fecha_atencion)

print(
    f"El rango de fechas de diagnosticos acotado es entre {fecha_min_diags_acotada} y "
    f" {fecha_max_diags_acotada}"
)

print(f"La base de diagnosticos acotados tiene {df_diagnosticos_acotado.shape[0]} consultas")

sns.histplot(df_diagnosticos_acotado["fecha_atencion"], label="Consultas")
sns.histplot(df_procedimientos["fecha_realizacion"], label="Procedimientos")
plt.legend()
plt.show()

Con el filtro aplicado se observa que ambas bases se encuentran en el mismo periodo temporal.
Finalmente, en ambas bases se eliminarán los registros que carezcan de una fecha de procedimiento.

In [ ]:
df_diagnosticos_acotado = df_diagnosticos_acotado.dropna(subset="fecha_atencion")
df_procedimientos = df_procedimientos.dropna(subset="fecha_realizacion")

print(f"La base de datos de consultas acotada tiene {df_diagnosticos_acotado.shape[0]} consultas")
print(f"La base de datos de procedimeintos tiene {df_procedimientos.shape[0]} procedimientos")

Por lo tanto, ambas bases se dejaron en un mismo rango temporal. La base de diagnósticos quedó
con 91916 consultas (de 265441, diferencia de 173525), y la base de procedimientos con 39472 (de 39484, 
diferencia de 12).

## 2 Obtención de insights más importantes de consultas

Una vez exportado el análisis se pueden responder las preguntas planteadas al inicio de la
sección.

In [ ]:
cantidad_consultas_por_anio = df_diagnosticos.groupby("ano").size()
cantidad_consultas_por_anio.name = "consultas_his"

print(f"La cantidad de consultas en la base de datos es: \n{cantidad_consultas_por_anio}\n")

In [ ]:
# Obtiene las consultas por especialidad
consultas_por_especialidad = (
    df_diagnosticos.groupby(["ano", "nombre_especialidad"])
    .size()
    .reset_index(name="cantidad_de_consultas")
)
consultas_por_especialidad.to_excel("../data/interim/consultas_por_especialidad.xlsx")

In [ ]:
AGRUPACIONES_ESPECIALIDAD = {
    "Broncopulmonar": [
        "AVNIA",
        "BRONCOPULMONAR ADULTO INT",
        "BRONQUIECTACIAS",
        "COMPIN",
        "DERRAME PLEURAL",
        "EPOC TIOTROPIO",
        "ENFERMEDADES PROFESIONALES",
        "EX. BRONCOSCOPIA",
        "FIBROSIS PULMONAR",
        "GES ASMA",
        "GES EPOC",
        "GES FIBROSIS QUISTICA",
        "HIPERTENSION PULMONAR",
        "INGRESO PROGRAMA DE OXIGENO",
        "OXIGENOTERAPIA",
        "PESQUISA CANCER PULMONAR",
        "PRIORITARIO BRONCOPULMONAR",
        "PULMON REUMATOLÓGICO",
        "TABACO GRUPAL",
        "TABACO INDIVIDUAL",
        "TBC",
    ],
    "Cardiocirugia": [
        "CARDIOCIRUGIA",
        "CARDIOPATIA CONGENITA",
        "TRASPLANTE CARDIACO",
        "PATOLOGÍA DE LA AORTA TORÁCICA-MARFAN",
    ],
    "Cirugia de Torax": [
        "CIRUGIA DE TORAX",
        "ONCOLOGIA",
        "TRASPLANTE PULMONAR",
    ],
    "Unidad del Sueno": [
        "UNIDAD DE SUEÑO",
        "UNIDAD DE SUEÑO-OTORRINOLARINGOLOGO",
    ],
    "Cardiologia": [
        "ARRITMIA",
        "CARDIOLOGIA",
        "EX. ECOCARDIO URGENCIA",
        "EX. ECOCARDIOGRAMA",
        "EX. HOLTER CONGENITOS INT",
        "EX. TEST DE ESFUERZO CONGENITO",
        "GES MARCAPASO",
        "GES MARCAPASO PRE QUIRÚRGICO",
        "DESFIBRILADORES / RESINCRONIZADORES",
        "ELECTROFISIOLOGIA",
    ],
    "Cuidados Paliativos": ["GES CUIDADOS PALIATIVOS"],
}

sin_enfermera = df_diagnosticos[
    ~df_diagnosticos["nombre_especialidad"].str.contains("ENFERMERA")
].copy()
sin_enfermera["especialidad_agrupada"] = sin_enfermera["nombre_especialidad"].replace(
    AGRUPACIONES_ESPECIALIDAD["Broncopulmonar"], "Broncopulmonar"
)

In [ ]:
sin_enfermera.groupby("ano")["nombre_diagnostico"].value_counts().to_excel("prueba_2.xlsx")

In [ ]:
sin_enfermera.query("nombre_especialidad == 'INMUNODEFICIENCIA'")[["codigo_diagnostico", "nombre_diagnostico"]].value_counts().head(30)

In [ ]:
sin_enfermera.query("especialidad_agrupada == 'Broncopulmonar'").groupby(["ano"]).size()

A modo de control, se contrastará la base de datos de consultas con la base de datos REM (REM07 (Consultas Medicas) + REM32 (Consultas Medicas Telefonicas)). Esto, ya que permite ver que tan fidedigna
es la base de datos ambulatoria. La base de datos REM tiene la siguiente cantidad de consultas:

In [ ]:
resumen_rem = pd.read_csv("../data/external/resumen_datos_rem.csv").set_index("year")
display(resumen_rem)

In [ ]:
comparacion_bd_rem = (
    pd.merge(cantidad_consultas_por_anio, resumen_rem, left_index=True, right_index=True)
    .iloc[:, [0, -1]]
    .reset_index()
)

comparacion_bd_rem.columns = ["year", "consultas_bd", "consultas_rem"]
comparacion_bd_rem["dif"] = comparacion_bd_rem.consultas_bd - comparacion_bd_rem.consultas_rem

promedio_diferencias_bd_rem = comparacion_bd_rem.dif.mean()
minimo_diferencias_bd_rem = comparacion_bd_rem.dif.min()
maximo_diferencias_bd_rem = comparacion_bd_rem.dif.max()

print(f"La comparacion de la base de datos con REM es: \n{comparacion_bd_rem}\n")
print(
    f"Hay una diferencia promedio de {promedio_diferencias_bd_rem:.0f} consultas entre la base "
    f"de datos y REM, con un minimo de {minimo_diferencias_bd_rem} y un maximo de "
    f"{maximo_diferencias_bd_rem} consultas. La distribucion de la diferencia entre ambas "
    f"bases es:"
)

sns.boxplot(comparacion_bd_rem["dif"])
plt.show()

## 3. Diagnosticos mas relevantes entre 2016 y 2023

Posteriormemnte, se quieren seleccionar los 20 diagnósticos más frecuentes para cada uno de los
años. Luego, se generará una lista conteniendo solamente los diagnósticos únicos de las listas
anteriores.

In [ ]:
conteo_consultas_por_diags = (
    df_diagnosticos.groupby("ano")["codigo_diagnostico"].value_counts().reset_index()
)

diags_mas_frecuentes_por_anio = sorted(
    conteo_consultas_por_diags.groupby("ano").head(20)["codigo_diagnostico"].unique()
)

print(f"Los diagnosticos mas frecuentes son: \n{diags_mas_frecuentes_por_anio}")
print(f"> {len(diags_mas_frecuentes_por_anio)} diagnosticos")

In [ ]:
consultas_totales_mas_relevantes = conteo_consultas_por_diags[
    conteo_consultas_por_diags["codigo_diagnostico"].isin(diags_mas_frecuentes_por_anio)
]

In [ ]:
display(consultas_totales_mas_relevantes)

In [ ]:
diccionario_resultado = {
    "conteo_consultas": conteo_consultas_por_diags,
    "conteo_consultas_mas_rel": consultas_totales_mas_relevantes,
}

with pd.ExcelWriter("../data/interim/diagnosticos_mas_relevantes_ambulatorios.xlsx") as file:
    for nombre_hoja, df_a_guardar in diccionario_resultado.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)

## 4. Análisis de Procedimientos por Diagnósticos

En este apartado se quieren responder las siguientes preguntas:

- ¿Cuál es la canasta de procedimientos que deben tener los pacientes para cada diagnósticos?
Por ejemplo: ¿Cuántas tomografías debe tener un paciente que tenga una insuficiencia aórtica?

- ¿Existe alguna canasta de procedimientos distinta para pacientes con un mismo diagnóstico?


Uno de los problemas que existe al responder esta pregunta, es que la base de datos estadísticos/procedimientos
carece de un diagnóstico para el paciente. 

El problema anterior puede ser resuelto utilizando la base de datos de consultas ambulatoria. La
lógica de la resolución es la siguiente:

1. Se utilizará la base de datos de procedimientos como base principal.
2. La base anterior será filtrada dejando solamente pacientes que hayan tenido una consulta ambulatoria.
3. Luego, a cada dulpa de paciente-procedimiento se la asignará un diagnóstico. Para asignar el
diagnóstico se seguiran los siguientes pasos:
    1. Se obtendrá la fecha en que el paciente se realizó el procedimiento
    2. Desde la base de consultas ambulatorias, se encontrará la consulta más próxima a la fecha
    del procedimiento.
    3. Una vez encontrada la consulta más próxima al procedimiento, se asignará el diagnóstico
    asociado a tal consulta.

Una vez aplicado los pasos anterior, se obtendrá una base de datos donde se relaciona el diagnóstico
y los procedimientos realizados.

### 3.1 Obtención de pacientes con procedimientos Y consultas

In [ ]:
pacientes_en_consultas = df_diagnosticos["id_paciente"].unique()
pacientes_en_procedimientos = df_procedimientos["id_paciente"].unique()


pacientes_con_proced_y_consultas = df_procedimientos[
    df_procedimientos["id_paciente"].isin(pacientes_en_consultas)
]

lista_pacientes_con_proced_y_consultas = pacientes_con_proced_y_consultas["id_paciente"].unique()

print(f"> Pacientes en Consultas: {len(pacientes_en_consultas)}")
print(f"> Pacientes en Procedimientos: {len(pacientes_en_procedimientos)}")
print(f"> Pacientes con Procedimientos y Consultas: {len(lista_pacientes_con_proced_y_consultas)}")

Por lo tanto, existen **2165** (14737 - 12572) pacientes que tuvieron un procedimiento pero ninguna
consulta en el Instituto Nacional del Tórax. Tales pacientes deben ser filtrados, ya que nunca
se les podrá asignar un diagnóstico.

In [ ]:
pacientes_con_proced_y_sin_consultas = df_procedimientos[
    ~df_procedimientos["id_paciente"].isin(pacientes_en_consultas)
]

### 3.2 Obtención de diagnóstico para cada procedimiento realizado por un paciente

Una vez se han dejado solamente los pacientes a los que se les puede asignar un diagnóstico,
se debe proceder a asignar un diagnóstico. Para lo anterior, se deben obtener todas las sesiones
únicas de procedimientos para un paciente.

In [ ]:
consultas_pacientes_con_proced = df_diagnosticos[
    df_diagnosticos["id_paciente"].isin(lista_pacientes_con_proced_y_consultas)
]

In [ ]:
proced_con_diagnosticos = build_features.asignar_diagnosticos_a_todos_los_procedimientos(
    pacientes_con_proced_y_consultas, consultas_pacientes_con_proced
)

### 3.3 Obtención de procedimientos por paciente

Una vez obtenido el diagnóstico para cada uno de los procedimientos, es posible responder a las
siguiente preguntas planteada previamente:

¿Cómo ha sido la canasta de procedimientos que han tenido los pacientes según su diagnóstico?


In [ ]:
cantidad_proceds_por_pacientes = build_features.obtener_cartera_de_procedimientos_por_diagnostico(
    proced_con_diagnosticos
)

In [ ]:
cantidad_proceds_por_pacientes.query("codigo_diagnostico == '17'")

Ahora, se obtuvo la cartera de procedimientos para pacientes que tuvieron algún procedimiento. Sin
embargo, es importante saber cuántos pacientes en total tuvieron consultas en el periodo analizado
(2020 y 2022).

In [ ]:
pacientes_unicos_consultas_procedimientos = (
    df_diagnosticos_acotado.groupby(["year", "codigo_diagnostico"])["id_paciente"]
    .nunique()
    .reset_index(name="cantidad_pacientes_distintos_total")
)

cantidad_proceds_por_pacientes = cantidad_proceds_por_pacientes.merge(
    pacientes_unicos_consultas_procedimientos, how="left", on=["year", "codigo_diagnostico"]
)

cantidad_proceds_por_pacientes["cantidad_proced_por_pacientes_total"] = (
    cantidad_proceds_por_pacientes["cantidad_procedimientos"]
    / cantidad_proceds_por_pacientes["cantidad_pacientes_distintos_total"]
)

cantidad_proceds_por_pacientes = cantidad_proceds_por_pacientes.merge(
    cie, how="left", left_on="codigo_diagnostico", right_on="Código"
)

In [ ]:
display(cantidad_proceds_por_pacientes)

In [ ]:
DIAGNOSTICOS_MAS_RELEVANTES = [
    "C33X",
    "C340",
    "C341",
    "C342",
    "C343",
    "C381",
    "C384",
    "C450",
    "C780",
    "C782",
    "D143",
    "D381",
    "E848",
    "I051",
    "I080",
    "I081",
    "I340",
    "I350",
    "I351",
    "I352",
    "I420",
    "I456",
    "I472",
    "I495",
    "I710",
    "I712",
    "J398",
    "J47X",
    "J679",
    "J841",
    "J848",
    "J849",
    "J860",
    "J869",
    "J90X",
    "J931",
    "J955",
    "J980",
    "M348",
    "Q211",
    "Q231",
    "Q676",
    "T820",
    "T821",
    "Z450",
]

canasta_proceds_diags_mas_relevantes = cantidad_proceds_por_pacientes[
    cantidad_proceds_por_pacientes["codigo_diagnostico"].isin(DIAGNOSTICOS_MAS_RELEVANTES)
]

In [ ]:
with pd.ExcelWriter("../data/interim/canasta_de_procedimientos_ambulatorios.xlsx") as file:
    cantidad_proceds_por_pacientes.to_excel(file, sheet_name="canasta_global", index=False)
    canasta_proceds_diags_mas_relevantes.to_excel(
        file, sheet_name="canasta_diags_mas_relevantes", index=False
    )

Los resultados indican la cantidad de procedimientos que se han realizado por un diagnóstico.
A modo de ejemplo, se discutirán los resultados para el diagnóstico "17":

- En primer lugar, se observa que se han realizado 6 procedimientos en total. Al ver el detalle,
se puede ver que 5 de esos procedimientos corresponden a una Tomografia Computarizada de Torax
y 1 a un Ecocardiograma bidimensional doppler color.

- Para el diagnóstico, existen 4 pacientes distintos atendidos en todo el periodo.

- Por lo tanto, para el diagnóstico "17" los pacientes se realizan en promedio 1.5 procedimientos.

- Si se observa la Tomografía Computarizada de Tórax, se observa que este tiene una proporcion de
1.25. **Esto indica que el 125% de los pacientes que son diagnosticados con "17" se deben realizar
una TAC de Tórax**. Por otro lado, el Ecocardiograma Bidimensional Doppler tiene una proporación
de 0.25. Lo anterior indica que sólo el 25% de los pacientes que son diagnosticados con "17"
se deben realizar un Ecocardiograma Bidimensional Doppler.

In [ ]:
procedimientos_de_diags_mas_relevantes = cantidad_proceds_por_pacientes[
    cantidad_proceds_por_pacientes["codigo_diagnostico"].isin(diags_mas_frecuentes_por_anio)
]

display(procedimientos_de_diags_mas_relevantes)

### 3.4 Cantidad de procedimientos realizados en el mismo día de una consulta

En este apartado se quiere responder a la siguiente pregunta:

- ¿Cuántas procedimientos fueron realizadas el mismo día de la consulta ambulatoria?
- De los procedimientos realizados después de una consulta ambulatoria, ¿Cuánto es el tiempo de
espera promedio? ¿Se puede saber por diagnóstico?

Para responder tales preguntas se utilizará la base de diagnósticos y procedimientos acotados.
Esto, ya que deben estar en un mismo periodo para hacer una comparación.

In [ ]:
proced_en_dia_de_consulta = build_features.obtener_procedimientos_en_dia_de_consulta(
    df_procedimientos, df_diagnosticos_acotado
)

In [ ]:
cantidad_proced_en_consulta = proced_en_dia_de_consulta.shape[0]

In [ ]:
print(f"> Procedimientos realizados en el mismo dia de la consulta: {cantidad_proced_en_consulta}")
print(f"> Procedimientos totales: {df_procedimientos.shape[0]}")

## 5. Obtención de procedimientos más frecuentes por unidad

La unidad de Hemodinamia es de alto interés para el Instituto Nacional del Tórax. Debido a lo anterior,
es necesario identificar cuales son los procedimientos realizados en esta especialidad.

Además, se quiere ver que exámenes son más frecuentes por cada una de las unidades del INT.

In [ ]:
conteo_hemodinamia = df_procedimientos.query(
    "unidad_que_la_realiza == 'HEMODINAMÍA'"
).glosa.value_counts()

display(conteo_hemodinamia)

En el resultado anterior se muestran todos los procedimientos de Hemodinamia realizado en el área
ambulatoria del Instituto Nacional del Tórax entre 2020 y 2022.

In [ ]:
procedimientos_mas_frecuentes = df_procedimientos.groupby(["unidad_que_la_realiza"])[
    "glosa"
].value_counts().reset_index(name="cantidad_procedimientos")

In [ ]:
display(procedimientos_mas_frecuentes)

In [ ]:
por_unidad = [
    procedimientos_mas_frecuentes.query("unidad_que_la_realiza == @unidad")
    .drop(columns="unidad_que_la_realiza")
    .rename(
        columns={
            "glosa": "Nombre de Procedimiento",
            "cantidad_procedimientos": "Cantidad de Procedimientos",
        }
    )
    for unidad in procedimientos_mas_frecuentes["unidad_que_la_realiza"].unique()
]

In [ ]:
visualize.add_dataframes_to_powerpoint(
    por_unidad,
    "../reports/presentacion.pptx",
    font_size=9,
    cell_width=12,
    cell_height=0.5,
    font_family="Open Sans",
    max_cell_characters=100,
)

Los resultados indican los procedimientos más frecuentes por cada subunidad del INT. La tabla
se encuentra resumida.

A modo de ejemplo, Anatomía Patología realiza Estudios Histopatologicas con Técnicas de Inmunohistoquímica
como procedimiento más frecuente.

In [ ]:
procedimientos_mas_frecuentes.to_excel(
    "../data/interim/procedimientos_mas_frecuentes.xlsx", index=False
)

## 5.1 Obtención de diagnósticos con la mayor cantidad de procedimientos de Broncoscopía

En este apartado se quiere responder la siguiente pregunta:

- ¿Cuáles son los diagnósticos más frecuentes que se realizan procedimientos de la unidad de
broncoscopía en el INT?

In [ ]:
PROCEDIMIENTOS_BRONCO = procedimientos_mas_frecuentes.query(
    "unidad_que_la_realiza == 'BRONCOSCOPÍA'"
).glosa.unique()

In [ ]:
canasta_diags_con_bronco = cantidad_proceds_por_pacientes[
    cantidad_proceds_por_pacientes["glosa"].isin(PROCEDIMIENTOS_BRONCO)
]

canasta_diags_con_bronco["glosa"] = canasta_diags_con_bronco["glosa"].str.strip()
canasta_diags_con_bronco = canasta_diags_con_bronco.sort_values(
    ["glosa", "cantidad_proced_por_pacientes", "cantidad_procedimientos"],
    ascending=[True, False, False],
)

In [ ]:
canasta_diags_con_bronco.head(10)

In [ ]:
canasta_diags_con_bronco.to_excel(
    "../data/interim/diags_con_procedimientos_de_bronco.xlsx", index=False
)